In [2]:
def yolo_to_corners(label, img_width, img_height):
    """
    Преобразует лейбл YOLO в координаты углов прямоугольника.

    :param label: Лейбл YOLO в формате [class_id, x_center, y_center, width, height]
    :param img_width: Ширина изображения в пикселях
    :param img_height: Высота изображения в пикселях
    :return: Кортеж с координатами углов прямоугольника (x_min, y_min, x_max, y_max)
    """
    class_id, x_center_norm, y_center_norm, width_norm, height_norm = label

    # Денормализация координат+

    x_center = x_center_norm * img_width
    y_center = y_center_norm * img_height
    width = width_norm * img_width
    height = height_norm * img_height

    # Вычисление координат углов
    x_min = int(x_center - width / 2)
    y_min = int(y_center - height / 2)
    x_max = int(x_center + width / 2)
    y_max = int(y_center + height / 2)

    return x_min, y_min, x_max, y_max

In [3]:
import os 
import cv2 as cv
import shutil

path_img_train = "F:/datasets/images/train/"
path_label_train = "F:/datasets/labels/train/"

path_img_test = "F:/datasets/images/test/"
path_label_test = "F:/datasets/labels/test/"

files_img = os.listdir(path_img_train)
files_label = os.listdir(path_img_train)


for i, filename in enumerate(files_img):
    img = cv.imread(path_img_train + filename)
    h, w = img.shape[:2]
    label_list = []

    with open(path_label_train + filename.replace(".jpg", ".txt"), 'r') as f:
        for line in f.readlines():
            label, x1, y1, x2, y2 = line.split()
            label_list.append([int(label), float(x1), float(y1), float(x2), float(y2)])

    
    for label in label_list:
        x_min, y_min, x_max, y_max = yolo_to_corners(label, w, h)
        print(x_min, y_min, x_max, y_max)
        cv.rectangle(img, (x_min, y_min),(x_max, y_max), (255,0,0),3)

    cv.imshow("image", img)


    while True:
        key = cv.waitKey(0) & 0xFF
        if key == ord('e'):
            print(f"Продолжение цикла. {i}")
            shutil.move(path_img_train + filename, "F:/datasets/images_corrent/train/" + filename)
            shutil.move(path_label_train + filename.replace(".jpg", ".txt"), "F:/datasets/labels_corrent/train/" + filename.replace(".jpg", ".txt"))
            break
        elif key == ord('q'):  
            print(f"Неправильная разметка. {i}")
            break

cv.destroyAllWindows()

318.29699800000003 = 0.4312967452574526 * 738
262 61 374 212
Продолжение цикла. 0
30.0 = 0.029296875 * 1024
0 489 60 563
425.000256 = 0.4150393125 * 1024
299 201 550 601
764.0 = 0.74609375 * 1024
639 107 888 474
Продолжение цикла. 1
647.2 = 0.63203125 * 1024
546 395 748 651
Продолжение цикла. 2
477.91999999999996 = 0.46671874999999996 * 1024
145 46 810 766
Продолжение цикла. 3
333.917572 = 0.4811492391930836 * 694
169 161 498 606
Продолжение цикла. 4
377.794738 = 0.5531401727672035 * 683
267 328 488 607
Продолжение цикла. 5
505.307328 = 0.4934641875 * 1024
13 0 996 390
Продолжение цикла. 6
354.32 = 0.346015625 * 1024
263 0 444 160
Продолжение цикла. 7
472.29304 = 0.5434902646720369 * 869
231 287 713 855
Продолжение цикла. 8
379.32 = 0.3704296875 * 1024
280 160 478 456
660.96 = 0.64546875 * 1024
552 393 769 637
Продолжение цикла. 9
232.56 = 0.227109375 * 1024
210 473 254 516
345.52 = 0.337421875 * 1024
331 298 359 337
382.16 = 0.373203125 * 1024
363 415 400 455
427.08 = 0.4170703125 * 1

KeyboardInterrupt: 